In [1]:
# Imports
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.affinity import translate

In [2]:
# Load centroid and flow data
centroids_gdf = gpd.read_file("../../data/europe/europe_centriods.gpkg")
flows_gdf = gpd.read_file("../../data/europe/europe_flows.gpkg")

In [3]:
# Select base point for legend
base_point = centroids_gdf[centroids_gdf["NAME_ENGL"] == "Czechia"].geometry.values[0]

In [4]:
# Create vertical legend points
legend_vertical = gpd.GeoDataFrame([
    {"geometry": base_point, "prichozi": 1001, "odchozi": 1000, "celkem": 2001},
    {"geometry": translate(base_point, xoff=0, yoff=-25000*3.75), "prichozi": 1000, "odchozi": 1001, "celkem": 2001}
], crs=centroids_gdf.crs)

print(legend_vertical)

                          geometry  prichozi  odchozi  celkem
0  POINT (4680695.035 2977518.749)      1001     1000    2001
1  POINT (4680695.035 2883768.749)      1000     1001    2001


In [5]:
# Create vertical legend points
legend_vertical2 = gpd.GeoDataFrame([
    {"geometry": translate(base_point, xoff=0, yoff=40000*3.75), "prichozi": 0, "odchozi": 1000, "celkem": 1000},
    {"geometry": translate(base_point, xoff=0, yoff=50000*3.75), "prichozi": 1000, "odchozi": 0, "celkem": 1000}
], crs=centroids_gdf.crs)

print(legend_vertical2)

                          geometry  prichozi  odchozi  celkem
0  POINT (4680695.035 3127518.749)         0     1000    1000
1  POINT (4680695.035 3165018.749)      1000        0    1000


In [6]:
# Create vertical legend points
legend_vertical3 = gpd.GeoDataFrame([
    {"geometry": translate(base_point, xoff=0, yoff=95000*3.75), "type": 1},
    {"geometry": translate(base_point, xoff=0, yoff=110000*3.75), "type": 2},
    {"geometry": translate(base_point, xoff=0, yoff=120000*3.75), "type": 3},
    {"geometry": translate(base_point, xoff=0, yoff=130000*3.75), "type": 4},
    {"geometry": translate(base_point, xoff=0, yoff=140000*3.75), "type": 5},
    {"geometry": translate(base_point, xoff=0, yoff=150000*3.75), "type": 6}
], crs=centroids_gdf.crs)

print(legend_vertical3)

                          geometry  type
0  POINT (4680695.035 3333768.749)     1
1  POINT (4680695.035 3390018.749)     2
2  POINT (4680695.035 3427518.749)     3
3  POINT (4680695.035 3465018.749)     4
4  POINT (4680695.035 3502518.749)     5
5  POINT (4680695.035 3540018.749)     6


In [7]:
# Create horizontal legend points
point_left   = translate(base_point, xoff=0, yoff=-50000*3.75)
point_middle = translate(base_point, xoff=25000*3.75, yoff=-50000*3.75)
point_right  = translate(base_point, xoff=100000*3.75, yoff=-50000*3.75)

In [8]:
# Get values from selected regions
total_kk  = centroids_gdf[centroids_gdf["NAME_ENGL"] == "Iceland"]["celkem"].values[0]
total_stc = centroids_gdf[centroids_gdf["NAME_ENGL"] == "Czechia"]["celkem"].values[0]
total_prg = centroids_gdf[centroids_gdf["NAME_ENGL"] == "Germany"]["celkem"].values[0]

In [9]:
# Find maximum and minimum values in flow data
max_flow = flows_gdf["value"].max()
min_flow = flows_gdf["value"].min()

print(f"Maximum flow value: {max_flow}")
print(f"Minimum flow value: {min_flow}")

Maximum flow value: 38178
Minimum flow value: 0


In [10]:
# Round total_kk and total_prg to the nearest thousand
kk_rounded = round(total_kk / 1000) * 1000
prg_rounded = round(total_prg / 1000) * 1000

In [11]:
# Compute square roots
sqrt_kk = np.sqrt(kk_rounded)
sqrt_prg = np.sqrt(prg_rounded)

In [12]:
# Calculate the average of the square roots
avg_sqrt = (sqrt_kk + sqrt_prg) / 2

In [13]:
# Square the average and round to the nearest thousand
middle = round((avg_sqrt ** 2) / 1000) * 1000

In [14]:
# Print for verification
print("Rounded total_kk:", kk_rounded)
print("Rounded total_prg:", prg_rounded)
print("Smoothed total_stc:", middle)

Rounded total_kk: 3000
Rounded total_prg: 234000
Smoothed total_stc: 72000


In [15]:
# horizontal legend
legend_horizontal = gpd.GeoDataFrame([
    {"geometry": translate(base_point, xoff=0, yoff=-50000*3.75),   "celkem": kk_rounded},
    {"geometry": translate(base_point, xoff=40000*3.75, yoff=-40600*3.75), "celkem": 70000},
    {"geometry": translate(base_point, xoff=100000*3.75, yoff=-30200*3.75),  "celkem": 250000}
], crs=centroids_gdf.crs)
print(legend_horizontal)

                          geometry  celkem
0  POINT (4680695.035 2790018.749)    3000
1  POINT (4830695.035 2825268.749)   70000
2  POINT (5055695.035 2864268.749)  250000


In [16]:
# Created lines
full_line = LineString([point_left, point_right])
full_line_shifted = translate(full_line, xoff=0, yoff=-25000*3.75)

In [17]:
# Add trimmed segments
point_middle_25 = translate(base_point, xoff=50000*3.75, yoff=-50000*3.75)
line_50l = LineString([point_left, point_middle_25])
line_50r = LineString([point_middle_25, point_right])

point_middle_25 = translate(base_point, xoff=25000*3.75, yoff=-50000*3.75)
line_25l = LineString([point_left, point_middle_25])
line_25r = LineString([point_middle_25, point_right])

point_middle_75 = translate(base_point, xoff=75000*3.75, yoff=-50000*3.75)
line_75l = LineString([point_left, point_middle_75])
line_75r = LineString([point_middle_75, point_right])

In [18]:
# Translate points
line_50l = translate(line_50l, xoff=0, yoff=-25000*3.75)
line_50r = translate(line_50r, xoff=0, yoff=-25000*3.75)
line_25l = translate(line_25l, xoff=0, yoff=-50000*3.75)
line_25r = translate(line_25r, xoff=0, yoff=-50000*3.75)
line_75l = translate(line_75l, xoff=0, yoff=-75000*3.75)
line_75r = translate(line_75r, xoff=0, yoff=-75000*3.75)

In [19]:
# 5. Create geodatafram
legend_lines = gpd.GeoDataFrame([
    {"geometry": line_50l, "nazev": "50l", "typ": "l", "label": "line_low_1", "celkem": 40000},
    {"geometry": line_50r, "nazev": "50r", "typ": "r", "label": "line_low_1", "celkem": 40000},
    {"geometry": line_25l, "nazev": "25l", "typ": "l", "label": "line_low_2", "celkem": 5000},
    {"geometry": line_25r, "nazev": "25r", "typ": "r", "label": "line_low_2", "celkem": 5000},
    {"geometry": line_75l, "nazev": "75l", "typ": "l", "label": "line_low_3", "celkem": 1000},
    {"geometry": line_75r, "nazev": "75r", "typ": "r", "label": "line_low_3", "celkem": 1000},
], crs=legend_horizontal.crs)

In [20]:
# 5. Save all to a single GeoPackage
output_path = "../../data/europe/europe_legend.gpkg"
legend_vertical.to_file(output_path, layer="legend_vertical", driver="GPKG")
legend_vertical2.to_file(output_path, layer="legend_vertical2", driver="GPKG")
legend_vertical3.to_file(output_path, layer="legend_vertical3", driver="GPKG")
legend_horizontal.to_file(output_path, layer="legend_horizontal", driver="GPKG")
legend_lines.to_file(output_path, layer="legend_lines", driver="GPKG")